In [1]:
import load
import numpy as np
import matplotlib
matplotlib.use("Qt5Agg")
import matplotlib.pyplot as plt
import tree
#import makegal
from galaxymodule import make_gal, rd_GM

In [2]:
cd ~/Work/data/HAGN/

/home/hoseung/Work/data/HAGN


In [3]:
from general import defaults
dfl = defaults.Default()
nout = 782#782 # 312
s = load.sim.Sim(nout=nout)
gcat = tree.halomodule.Halo(nout=nout, is_gal=True)
gg = rd_GM.Gal(nout, catalog=gcat.data[4].copy(), info=s.info)
gg.debug=False
make_gal.mk_gal(gg, convert_time=True)


[sim._hilbert_cpulist] No AMR instance,
[sim._hilbert_cpulist] Loading one...
An AMR instance is created
 Sim
An AMR instance is created
 Sim
Simulation set up.
[Halo.load_info] loading info
[Halo.load_info] nout = 782, base =./
info <load.info.Info object at 0x7f21d3ec7390>
File Not Found: ./halo/HAL_00782/hal_dms_0000005
No DM data loaded
File Not Found: ./GalaxyMaker/CELL_00782/gal_cells_0000005
No CELL data loaded


True

In [11]:
import importlib

In [14]:
importlib.reload(qmc)

<module 'galaxymodule.quick_mock' from '/home/hoseung/Work/pyclusterevol/galaxymodule/quick_mock.py'>

In [15]:
from galaxymodule import quick_mock as qmc
MockSED = qmc.Simplemock(repo=dfl.dir_repo+'sed/')

In [16]:
#gg.star.Flux_u= MockSED.get_flux(star=gg.star, filter_name='u')
#gg.star.Flux_g= MockSED.get_flux(star=gg.star, filter_name='g')
gg.star.Flux_r= MockSED.get_flux(star=gg.star, filter_name='r')
#gg.star.Flux_i= MockSED.get_flux(star=gg.star, filter_name='i')
#gg.star.Flux_z= MockSED.get_flux(star=gg.star, filter_name='z')

# unit: erg/cm^2/Hz ....?

In [7]:
print(np.log10(gg.meta.mstar))

11.5624991756


In [17]:
np.mean(gg.star.Flux_r)

6.6083688602822799e+35

In [10]:
np.log10(np.mean(gg.star["m"]))

6.5436572630898517

In [18]:
total_flux = sum(gg.star.Flux_r)
print(total_flux)

6.90138393555e+40


In [56]:
Lum_sun = 3.826e33 # ergs/s

mgal = 1e11

pivot_lambda = 6175 # A

Lum_dist = 400 #400 # Mpc
d_lum_10p = 3.0857e19 # lumminonsity distance of 10pc in cm
speed_of_light = 3e18 # angstrom / sec
kpc_to_cm = 3.0857e21
ldcm = Lum_dist * kpc_to_cm * 1e3
s_inv = 1./(4*np.pi * ldcm*ldcm)

In [74]:
SED_r = MockSED.SEDs[3][190][500]
tot_flux = Lum_sun * mgal * SED_r * s_inv / 1e-17 #(5*kpc_to_cm)**2
# erg/s/cm2 / Angstrom

# The only difference from Ji is the missing 1e-2. 

# Now pass through filter and calculate mag

In [71]:
MockSED.filters["lambda"][130]

6230.0

In [94]:
d_l_filter = 1100 # rough width of r band.
print(d_l_filter)

1100


In [72]:
# Almost perfect tarnsperancy.
MockSED.filters["r"][130]

0.98046800000000001

In [102]:
filtered_flux = tot_flux / (0.98 * d_l_filter)# * pivot_lambda
# Don't understand why it's a division, not multiplication. 

In [110]:
# flux to integrated magnitude. 
# pivot lambda come in here, not before. 
Mag_abs = -2.5 * np.log10(filtered_flux) \
        - 5* np.log10(pivot_lambda) \
        + 2.5*np.log10(speed_of_light) \
        - 48.60 #\

apparent_mag = Mag_abs + 5*np.log10(Lum_dist * 1e6/1e1)
print("around -20? : ", apparent_mag)

around -20? :  15.4927164966


Why does it work..???

anyways,, let's make 2d mock image to see if it has a realistic surface brightness for a given observation condition. 

** smaller pixel scale = less photon = worse magnitude limit -> lower signal to noise.. 
But the observed objects apparent magnitude shouldn't change. 

In [126]:
plate_scale = 0.12 # second per pixel on a side.
gal_size = 10
npix = gal_size/plate_scale
Surface_Brightness = apparent_mag + 2.5*np.log10(npix**2) + 2.5*np.log10(plate_scale * plate_scale)
# If I assume all flux uniformly distributes over 50 x 50 pixels. 
# Surface brightness is brighter with larger (npix * plate_scale)
# If the galaxy is larger and more diffuse, for a fixed plate scale npix is higher. 
# If the galaxy size if fixed, varying plate_scale has no effect on the SB. 

In [127]:
Surface_Brightness

20.492716496616396

# idl

per pixel. 

SED * mass of each star * Lum_sun * 1e-2 * inv_s , where inv_s 


Still don't know what 1e-2 is for. 


Then, convolve with 

In [25]:
- 2.5 * np.log10(total_flux/Lum_sun) \
           - 5. * np.log10(6175) \
           + 2.5 * np.log10(speed_of_light)# -48.6

9.0991402698501389

In [19]:
(-21.1 + 18.6921 + 48.6)/2.5# = np.log10(c)

18.47684

In [26]:
np.log10(speed_of_light)

18.477121254719663

In [27]:
plt.plot(MockSED.sed_wavelength[:400], np.log10(MockSED.SEDs[0][40][:400]+0.1))
plt.show()

In [21]:
speed_of_light = 3e18 # angstrom / sec

In [39]:
Mag_ab = -2.5*np.log10(total_flux) -5 * np.log10(pivot_lambda) +2.5*np.log10(speed_of_light) - 48.60

In [28]:
plt.hist(np.log10(MockSED.sed_wavelength[1:] - MockSED.sed_wavelength[:-1]))
#plt.hist((MockSED.sed_wavelength[1:] - MockSED.sed_wavelength[:-1])/MockSED.sed_wavelength[:-1])
plt.show()

In [40]:
Mag_ab

-34.558346188750768

In [53]:
gg.star.Flux_r # 

array([ 0.29036335,  0.29036335,  0.29036335, ...,  0.20791595,
        0.20735446,  0.20946372])

In [51]:
fmap = qmc.flux2mag(gg.star.Flux_r, gg, plate_scale=0.1)

78 78


/home/hoseung/Work/pyclusterevol/galaxymodule/quick_mock.py:205: RuntimeWarning: divide by zero encountered in log10
  


In [52]:
plt.imshow(fmap)
plt.show()

1. Shouldn't depend on plate_scale change. 
2. total flux and magnutude must be constatnt

In [44]:
fmap[fmap < 100].max()

25.634454517914968

In [42]:
np.nan_to_num()

1.7976931348623157e+308

In [29]:
qmc.draw(gg, x1="x", x2="y", suffix="face")
qmc.draw(gg, x1="x", x2="z", suffix="edge")